In [39]:
import pandas as pd
from os import listdir

In [48]:
all_files = []
path = 'province_houses/'
for filename in listdir(path):
    if 'houses' in filename and 'csv' in filename:
        df = pd.read_csv(path + filename)
        df['source'] = filename.split('.')[0]
        all_files.append(df)

In [49]:
result = pd.concat(all_files)

In [51]:
result.to_csv('houses_final.csv')